In [1]:
pip install PyPDF2

In [ ]:
import PyPDF2
import json
import re

pdf_file = r"C:\Users\hp\Downloads\Python Snippet Question Bank.pdf"
output_json = "data/python_easy.json"


def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                # Remove formatting artifacts: multiple newlines, spaces
                clean_text = re.sub(r"\s+", " ", page_text)
                text += clean_text + "\n"
    return text


def parse_mcqs(text):
    # Updated regex: flexible line breaks, supports cleaner text
    pattern = re.compile(
        r"(\d+)\.\s*(.*?)\s*A\.\s*(.*?)\s*B\.\s*(.*?)\s*C\.\s*(.*?)\s*D\.\s*(.*?)\s*Answer:\s*([A-D])",
        re.DOTALL | re.IGNORECASE
    )

    questions = []
    for match in pattern.finditer(text):
        q_num = int(match.group(1))
        q_text = match.group(2).strip()
        a = match.group(3).strip()
        b = match.group(4).strip()
        c = match.group(5).strip()
        d = match.group(6).strip()
        correct = match.group(7).strip().upper()

        questions.append({
            "number": q_num,
            "question_text": q_text,
            "option_a": a,
            "option_b": b,
            "option_c": c,
            "option_d": d,
            "correct_option": correct,
            "explanation": ""
        })

    return questions


# Run Extraction
text = extract_text_from_pdf(pdf_file)
mcqs = parse_mcqs(text)

# Save to JSON ensuring clean output
def clean_json_data(data):
    for item in data:
        for key in item:
            if isinstance(item[key], str):
                item[key] = re.sub(r"\s+", " ", item[key]).strip()
    return data

cleaned_mcqs = clean_json_data(mcqs)

with open(output_json, "w", encoding="utf-8") as f:
    json.dump(cleaned_mcqs, f, indent=4, ensure_ascii=False)

print(f"Extracted {len(cleaned_mcqs)} cleaned questions into {output_json}")

PermissionError: [Errno 13] Permission denied: '/python_easy.json'